Import libraries

In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import geocoder # import geocoder

Read df from csv

In [9]:
neighborhoods = pd.read_csv('data_l.csv')
neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Get geographical coordinates of Toronto, Canada

In [43]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="yyz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


Map creation

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get the top 100 venues that are in Rouge and Malvern within a radius of 500 meters.

Get coordinates of the 1st neighborhood on list (Rouge, Malvern)

In [27]:
neighborhoods.loc[0, 'Neighborhood']

neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.8066863, -79.19435340000003.


Set your cridentials and parameters for Foursquare

In [13]:
CLIENT_ID = 'input your id' # your Foursquare ID
CLIENT_SECRET = 'input your key' # your Foursquare Secret
VERSION = '20190602' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G5EEM11JWEGK0MMB5XMPYKOCOYQ145BKPFDNGNKNZ3DVF0C1
CLIENT_SECRET:Q2GHHTGNZ2JSOJLBBDS4ICW43Y51CN3UI2N33BATC0XTXVMM


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf4085df594df57eb7fb259'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.18812958073043},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'di

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


Let's run the above function on each neighborhood of Central Toronto and create a new dataframe called toronto_venues

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
central_toronto_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
toronto_venues = getNearbyVenues(names=central_toronto_data['Neighborhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville


In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(109, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,31,31,31,31,31,31
Davisville North,9,9,9,9,9,9
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",14,14,14,14,14,14
"Forest Hill North, Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",3,3,3,3,3,3
North Toronto West,20,20,20,20,20,20
Roselawn,1,1,1,1,1,1
"The Annex, North Midtown, Yorkville",24,24,24,24,24,24


Analyze Each Neighborhood in Toronto Central

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_onehot.shape

(109, 60)

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.064516,0.00,0.000000,0.064516,0.000000,0.096774,0.032258,0.032258,0.00,0.000000,0.032258,0.0,0.00,0.032258,0.032258,0.032258,0.00,0.00,0.000000,0.000000,0.032258,0.064516,0.00,0.000000,0.000000,0.000000,0.00,0.032258,0.032258,0.064516,0.000000,0.000000,0.00,0.064516,0.00,0.096774,0.032258,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.064516,0.000000,0.032258,0.032258,0.00,0.000000,0.000000,0.00
1,Davisville North,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.0,0.00,0.000000,0.000000,0.111111,0.00,0.00,0.000000,0.222222,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.111111,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.071429,0.00,0.000000,0.000000,0.071429,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.071429,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.00
3,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.150000,0.100000,0.000000,0.050000,0.050000,0.000000,0.05,

In [34]:
toronto_grouped.shape

(9, 60)

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.10
1        Dessert Shop  0.10
2  Italian Restaurant  0.06
3         Pizza Place  0.06
4          Restaurant  0.06


----Davisville North----
              venue  freq
0             Hotel  0.22
1              Park  0.11
2  Asian Restaurant  0.11
3    Breakfast Spot  0.11
4    Sandwich Place  0.11


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                 venue  freq
0                  Pub  0.14
1          Coffee Shop  0.14
2  American Restaurant  0.07
3     Sushi Restaurant  0.07
4         Liquor Store  0.07


----Forest Hill North, Forest Hill West----
                 venue  freq
0                Trail  0.25
1                 Park  0.25
2     Sushi Restaurant  0.25
3        Jewelry Store  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1             Bus Line  0.33
2          Swim School  0.33
3  American Restaura

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Restaurant,Greek Restaurant,Farmers Market
1,Davisville North,Hotel,Sandwich Place,Clothing Store,Asian Restaurant,Food & Drink Shop,Breakfast Spot,Gym,Park,Fried Chicken Joint,Garden
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,American Restaurant,Pizza Place,Bagel Shop,Vietnamese Restaurant,Sushi Restaurant,Supermarket,Sports Bar,Light Rail Station
3,"Forest Hill North, Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Park,Yoga Studio,Diner,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
4,Lawrence Park,Swim School,Park,Bus Line,Yoga Studio,Diner,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop


Cluster Neighborhoods

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 3, 2, 0, 1, 0])

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = central_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Swim School,Park,Bus Line,Yoga Studio,Diner,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Sandwich Place,Clothing Store,Asian Restaurant,Food & Drink Shop,Breakfast Spot,Gym,Park,Fried Chicken Joint,Garden
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Gift Shop,Mexican Restaurant,Park,Fast Food Restaurant,Diner,Dessert Shop
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Restaurant,Greek Restaurant,Farmers Market
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Summer Camp,Playground,Restaurant,Yoga Studio,Diner,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,0,Hotel,Sandwich Place,Clothing Store,Asian Restaurant,Food & Drink Shop,Breakfast Spot,Gym,Park,Fried Chicken Joint,Garden
2,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Gift Shop,Mexican Restaurant,Park,Fast Food Restaurant,Diner,Dessert Shop
3,Central Toronto,0,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Restaurant,Greek Restaurant,Farmers Market
5,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Pizza Place,Bagel Shop,Vietnamese Restaurant,Sushi Restaurant,Supermarket,Sports Bar,Light Rail Station
8,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Park,Pub,Liquor Store,Cosmetics Shop,Jewish Restaurant,American Restaurant
